In [ ]:
# --- toy "search" over a tiny in-memory corpus ---
DOCS = {
    "germany_gdp": "Germany GDP in 2023 was about 4.4 trillion USD (nominal).",
    "france_gdp":  "France GDP in 2023 was about 3.1 trillion USD (nominal).",
    "heidelberg":  "Heidelberg is a university town on the Neckar River in southwest Germany."
}

def search_tool(query: str) -> str:
    q = query.lower()
    # naive keyword lookup
    if "germany" in q and "gdp" in q: return DOCS["germany_gdp"]
    if "france"  in q and "gdp" in q: return DOCS["france_gdp"]
    if "heidelberg" in q:             return DOCS["heidelberg"]
    return "No result found."

def calculator_tool(expr: str) -> str:
    # very small safe parser: only digits and +-*./()
    import re
    if not re.fullmatch(r"[0-9\.\+\-\*\/\(\) ]+", expr):
        return "Calculator error: invalid expression."
    try:
        return str(eval(expr))
    except Exception as e:
        return f"Calculator error: {e}"

def summarizer_tool(text: str, max_words: int = 18) -> str:
    words = text.split()
    return " ".join(words[:max_words]) + ("…" if len(words) > max_words else "")
